In [ ]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()  
prefix = 'pascalvoc-segmentation'

In [ ]:
s3_train_data = 's3://{}/{}/input/train'.format(bucket, prefix)
s3_validation_data = 's3://{}/{}/input/validation'.format(bucket, prefix)
s3_train_annotation_data = 's3://{}/{}/input/train_annotation'.format(bucket, prefix)
s3_validation_annotation_data = 's3://{}/{}/input/validation_annotation'.format(bucket, prefix)
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

print(s3_train_data)
print(s3_validation_data)
print(s3_train_annotation_data)
print(s3_validation_annotation_data)

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(sess.boto_region_name, 'semantic-segmentation', repo_version='latest')

In [ ]:
from sagemaker import get_execution_role
 
role = get_execution_role()

seg = sagemaker.estimator.Estimator(container,
                                    role, 
                                    sagemaker_session = sess,
                                    train_instance_count = 1, 
                                    train_instance_type = 'ml.g4dn.xlarge',
                                    output_path = s3_output_location)

In [ ]:
seg.set_hyperparameters(backbone='resnet-50', 
                             algorithm='fcn',              
                             use_pretrained_model=True, 
                             num_classes=21,
                             epochs=30,
                             num_training_samples=1464) 

Now that the hyperparameters are setup, let us prepare the handshake between our data channels and the algorithm. To do this, we need to create the `sagemaker.session.s3_input` objects from our data channels. These objects are then put in a simple dictionary, which the algorithm uses to train.

In [ ]:
train_data = sagemaker.session.s3_input(s3_train_data, distribution='FullyReplicated', 
                                        content_type='image/jpeg', s3_data_type='S3Prefix')

validation_data = sagemaker.session.s3_input(s3_validation_data, distribution='FullyReplicated', 
                                        content_type='image/jpeg', s3_data_type='S3Prefix')

train_annotation = sagemaker.session.s3_input(s3_train_annotation_data, distribution='FullyReplicated', 
                                        content_type='image/png', s3_data_type='S3Prefix')

validation_annotation = sagemaker.session.s3_input(s3_validation_annotation_data, distribution='FullyReplicated', 
                                        content_type='image/png', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 
                 'validation': validation_data,
                 'train_annotation': train_annotation, 
                 'validation_annotation':validation_annotation}

We have our `Estimator` object, we have set the hyperparameters for this object and we have our data channels linked with the algorithm. The only remaining thing to do is to train the algorithm. The following command will train the algorithm. Training the algorithm involves a few steps. Firstly, the instances that we requested while creating the `Estimator` classes are provisioned and are setup with the appropriate libraries. Then, the data from our channels are downloaded into the instance. Once this is done, the training job begins. The provisioning and data downloading will take time, depending on the size of the data and the availability of the type of instances. Therefore it might be a few minutes before we start getting data logs for our training jobs. The data logs will also print out training loss on the training data, which is the pixel-wise cross-entropy loss as described in the algorithm papers. The data logs will also print out pixel-wise label accuracy and mean intersection-over-union (mIoU) on the validation data after a run of the dataset once or one epoch. These metrics measure the quality of the model under training.

Once the job has finished a "Job complete" message will be printed. The trained model can be found in the S3 bucket that was setup as `output_path` in the estimator.

In [ ]:
seg.fit(inputs=data_channels)

In [ ]:
seg_predictor = seg.deploy(initial_instance_count=1, instance_type='ml.c5.2xlarge')

In [ ]:
!wget -O test.jpg http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/105.horse/105_0016.jpg
filename = 'test.jpg'

Let us convert the image to bytearray before we supply it to our endpoint.

In [ ]:
import matplotlib.pyplot as plt
import PIL

im = PIL.Image.open(filename)
im.save(filename, "JPEG")

%matplotlib inline
plt.imshow(im)
plt.axis('off')

In [ ]:
seg_predictor.content_type = 'image/jpeg'
seg_predictor.accept = 'image/png'

with open(filename, 'rb') as image:
    img = image.read()
    img = bytearray(img)

response = seg_predictor.predict(img)

Let us display the segmentation mask.

In [ ]:
import PIL
from PIL import Image
import numpy as np
import io

num_classes = 21
mask = np.array(Image.open(io.BytesIO(response)))
plt.imshow(mask, vmin=0, vmax=num_classes-1, cmap='gray_r')
plt.show()

In [ ]:
seg_predictor.content_type = 'image/jpeg'
seg_predictor.accept = 'application/x-protobuf'
response = seg_predictor.predict(img)

results_file = 'results.rec'
with open(results_file, 'wb') as f:
    f.write(response)

In [ ]:
from sagemaker.amazon.record_pb2 import Record
import mxnet as mx

rec = Record()
recordio = mx.recordio.MXRecordIO(results_file, 'r')
protobuf = rec.ParseFromString(recordio.read())

values = list(rec.features["target"].float32_tensor.values)
shape = list(rec.features["shape"].int32_tensor.values)

In [ ]:
print(shape)
print(len(values))
mask = np.reshape(np.array(values), shape)

In [ ]:
pixel_probs = mask[0,:,0,0]
print(pixel_probs)
print(np.argmax(pixel_probs))

In [ ]:
print(mask.shape)

In [ ]:
seg_predictor.delete_endpoint()